In [1]:
import pandas as pd
import numpy as np
import random as rnd
import pickle
import os
import math
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import norm

In [2]:
df_encounter=pd.read_csv("csv/ENCOUNTERS.csv")
df_procedures=pd.read_csv("csv/procedures.csv")
df_careplan=pd.read_csv("csv/careplans.csv")
df_observations=pd.read_csv("csv/observations.csv")
df_payers=pd.read_csv("csv/payers.csv")

In [3]:
#Pre-processing (maintaining few relationships)

encounterid_to_conditioncode = {}
conditiondescription_to_code={}
conditioncode_to_description={}
encounterid_to_totalcost={}
encounterid_to_recoveredcost={}
encounterid_to_insurancecompany={}


for ind in df_encounter.index:
    
    if pd.isnull(df_encounter["REASONCODE"][ind]):
        pass
    else:
        encounterid_to_conditioncode[df_encounter["Id"][ind]]=df_encounter["REASONCODE"][ind]
        conditiondescription_to_code[df_encounter["REASONDESCRIPTION"][ind]]=df_encounter["REASONCODE"][ind]
        conditioncode_to_description[df_encounter["REASONCODE"][ind]]=df_encounter["REASONDESCRIPTION"][ind]
        encounterid_to_insurancecompany[df_encounter["Id"][ind]]=df_encounter["PAYER"][ind]
        encounterid_to_totalcost[df_encounter["Id"][ind]]=float(df_encounter["TOTAL_CLAIM_COST"][ind])
        encounterid_to_recoveredcost[df_encounter["Id"][ind]]=float(df_encounter["PAYER_COVERAGE"][ind])

In [4]:
#Extracting all the procedures used for every condition

procedurescount_ineverycondtion={}
procedurecode_to_description={}
proceduredescription_to_code={}
for ind in df_procedures.index:
    if pd.isnull(df_procedures["REASONCODE"][ind]):
        pass
    else:
        conditioncode=df_procedures["REASONCODE"][ind]
        procedurecode=df_procedures["CODE"][ind]
        procedurecode_to_description[df_procedures["CODE"][ind]]=df_procedures["DESCRIPTION"][ind]
        proceduredescription_to_code[df_procedures["DESCRIPTION"][ind]]=df_procedures["CODE"][ind]
        if conditioncode in list(procedurescount_ineverycondtion.keys()):
            if procedurecode in list(procedurescount_ineverycondtion.get(conditioncode,{}).keys()):
                procedurescount_ineverycondtion[conditioncode][procedurecode]=1+procedurescount_ineverycondtion[conditioncode][procedurecode]
            else:
                procedurescount_ineverycondtion[conditioncode][procedurecode]=1;
        else:
            tmp={}
            tmp[procedurecode]=1;
            procedurescount_ineverycondtion[conditioncode]=tmp


In [5]:
#Extracting all the most likely procedures for every condition

conditioncode_to_releventprocedurelist={}
for condition in list(procedurescount_ineverycondtion.keys()):
    max_frequency=0;
    
    for procedure in list(procedurescount_ineverycondtion[condition].keys()):
        if max_frequency<procedurescount_ineverycondtion[condition][procedure]:
            max_frequency=procedurescount_ineverycondtion[condition][procedure]        
            
    for procedure in list(procedurescount_ineverycondtion[condition].keys()):
        if procedurescount_ineverycondtion[condition][procedure]>=(max_frequency/3):
            tmp=conditioncode_to_releventprocedurelist.get(condition,[])
            tmp.append(procedure)
            conditioncode_to_releventprocedurelist[condition]=tmp

In [6]:
#Calculating expected cost for every procedures

procedurecode_to_cost={}

for ind in df_procedures.index:
    if pd.isnull(df_procedures["CODE"][ind]):
        pass
    else:
        tmp=procedurecode_to_cost.get(df_procedures["CODE"][ind],[])
        tmp.append(float(df_procedures["BASE_COST"][ind]))
        procedurecode_to_cost[df_procedures["CODE"][ind]]=tmp

for procedure in list(procedurecode_to_cost.keys()):
    tmp=procedurecode_to_cost[procedure]
    totalcost=0.0
    totalcount=0
    for cost in tmp:
        totalcost=totalcost+cost
        totalcount=totalcount+1
    procedurecode_to_cost[procedure]=totalcost/totalcount           

In [7]:
#Extracting all the most likely careplans for every condition

careplancode_to_description={}
conditioncode_to_releventcareplanlist={}
for ind in df_careplan.index:
    if pd.isnull(df_careplan["REASONCODE"][ind]):
        pass
    else:
        condition=df_careplan["REASONCODE"][ind]
        currlist=conditioncode_to_releventcareplanlist.get(condition,[])
        if df_careplan["CODE"][ind] not in currlist:
            currlist.append(df_careplan["CODE"][ind])
            conditioncode_to_releventcareplanlist[condition]=currlist
        careplancode_to_description[df_careplan["CODE"][ind]]=df_careplan["DESCRIPTION"][ind]

In [8]:
#Extracting all the observations recorded for every condition

observationdescription_to_code={}
observationcode_to_description={}
observationcount_ineverycondtion={}

for num,observationdescription in enumerate(list(df_observations["DESCRIPTION"].unique())):
    observationdescription_to_code[observationdescription]=num+1
    observationcode_to_description[num+1]=observationdescription
    
    
for ind in df_observations.index:
    if df_observations["ENCOUNTER"][ind] in encounterid_to_conditioncode:
        if df_observations["TYPE"][ind]=="numeric":
            condition=encounterid_to_conditioncode[df_observations["ENCOUNTER"][ind]]
            observationcode=observationdescription_to_code[df_observations["DESCRIPTION"][ind]]
            if condition in list(observationcount_ineverycondtion.keys()):
                if observationcode in list(observationcount_ineverycondtion.get(condition,{}).keys()):
                    observationcount_ineverycondtion[condition][observationcode]=1+observationcount_ineverycondtion[condition][observationcode]
                else:
                    observationcount_ineverycondtion[condition][observationcode]=1
            else:
                tmp={}
                tmp[observationcode]=1
                observationcount_ineverycondtion[condition]=tmp

In [9]:
#Calculating and storing all the relevent observations corresponding to every condition

conditioncode_to_releventobservationlist={}

for condition in list(observationcount_ineverycondtion.keys()):
    maxfrequency=0
    for observation in list(observationcount_ineverycondtion[condition].keys()):
        if maxfrequency<observationcount_ineverycondtion[condition][observation]:
            maxfrequency=observationcount_ineverycondtion[condition][observation]
            
    for observation in list(observationcount_ineverycondtion[condition].keys()):
        if observationcount_ineverycondtion[condition][observation]>=maxfrequency/2:
            tmp=conditioncode_to_releventobservationlist.get(condition,[])
            tmp.append(observation)
            conditioncode_to_releventobservationlist[condition]=tmp

In [10]:
#Extracting all the readings observed for every observations/Vitals corresponding to every condition

conditioncode_to_observationreading={}
for ind in df_observations.index:
    if df_observations["ENCOUNTER"][ind] in encounterid_to_conditioncode:
        condition=encounterid_to_conditioncode[df_observations["ENCOUNTER"][ind]]
        observation=observationdescription_to_code[df_observations["DESCRIPTION"][ind]]
        
        if observation in conditioncode_to_releventobservationlist[condition]:
            if condition in list(conditioncode_to_observationreading.keys()):
                if observation in list(conditioncode_to_observationreading[condition].keys()):
                    conditioncode_to_observationreading[condition][observation].append(df_observations["VALUE"][ind])
                else:
                    tmplist=[]
                    tmplist.append(df_observations["VALUE"][ind])
                    conditioncode_to_observationreading[condition][observation]=tmplist
            else:
                tmpdict={}
                tmplist=[]
                tmplist.append(df_observations["VALUE"][ind])
                tmpdict[observation]=tmplist
                conditioncode_to_observationreading[condition]=tmpdict

In [11]:
#Calculating mean and deviation for every observation/Vital corresponding to every condition
#Normal distribution can be obtained from mean and deviation.

conditioncode_and_observationcode_to_mean={}
conditioncode_and_observationcode_to_deviation={}

for condition in list(conditioncode_to_observationreading.keys()):
    for observation in list(conditioncode_to_observationreading[condition].keys()):
        readinglist=conditioncode_to_observationreading[condition][observation]
        readinglist=np.array(readinglist)
        readinglist=readinglist.astype(np.float)
        totalrating=0.0
        totalcount=0.0
        for num in readinglist:
            totalrating=totalrating+num
            totalcount=totalcount+1
        mean=totalrating/totalcount
        deviation=0.0
        for reading in readinglist:
            deviation=deviation+(mean-reading)*(mean-reading)
        deviation=deviation/totalcount
        deviation=math.sqrt(deviation)
        
        currmeans=conditioncode_and_observationcode_to_mean.get(condition,{})
        currmeans[observation]=mean
        conditioncode_and_observationcode_to_mean[condition]=currmeans
        currdeviation=conditioncode_and_observationcode_to_deviation.get(condition,{})
        currdeviation[observation]=deviation
        conditioncode_and_observationcode_to_deviation[condition]=currdeviation

In [12]:
#Initialising every condition and corresponding procedures under every insurance company

insurancecompanyname_to_code={}
insurancecode_to_company={}
for ind in df_payers.index:
    insurancecompanyname_to_code[df_payers["NAME"][ind]]=df_payers["Id"][ind]
    insurancecode_to_company[df_payers["Id"][ind]]=df_payers["NAME"][ind]
    
insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio={}
for ind in df_payers.index:
    company=df_payers["Id"][ind]
    insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio[company]={}
    for condition in list(conditioncode_to_description.keys()):
        insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio[company][condition]={}
    for ind1 in df_procedures.index:
        if pd.isnull(df_procedures["REASONCODE"][ind1]):
            pass
        else:
            if df_procedures["REASONCODE"][ind1] in list(conditioncode_to_description.keys()):
                insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio[df_payers["Id"][ind]][df_procedures["REASONCODE"][ind1]][df_procedures["CODE"][ind1]]=[]

In [13]:
#Extracting and storing all the encounters of a procedure under corresponding condition and insurance company

for ind in df_procedures.index:
        if pd.isnull(df_procedures["REASONCODE"][ind]):
            pass
        else:
            if df_procedures["REASONCODE"][ind] in list(conditioncode_to_description.keys()):
                if df_procedures["ENCOUNTER"][ind] in list(encounterid_to_recoveredcost.keys()):
                    tmpval=encounterid_to_recoveredcost[df_procedures["ENCOUNTER"][ind]]/encounterid_to_totalcost[df_procedures["ENCOUNTER"][ind]]
                    insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio[encounterid_to_insurancecompany[df_procedures["ENCOUNTER"][ind]]][df_procedures["REASONCODE"][ind]][df_procedures["CODE"][ind]].append(tmpval)

In [14]:
#Calculating expected coverage of the procedure calculated above, under corresponding condition and insurance company

for company in list(insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio.keys()):
    for condition in list(insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio[company].keys()):
        for procedure in list(insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio[company][condition].keys()):
            tmplist=insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio[company][condition][procedure]
            expectedratio=0.0
            cnt=int(len(tmplist))
            if cnt!=0:
                for ratio in tmplist:
                    expectedratio=expectedratio+ratio
                expectedratio=expectedratio/cnt
            insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio[company][condition][procedure]=expectedratio

In [15]:
#Saving all the dictionaries as pickles for faster access

with open('pickles/condition_description_to_code.pickle', 'wb') as handle:
    pickle.dump(conditiondescription_to_code, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/condtioncode_to_listofcareplancodes.pickle', 'wb') as handle:
    pickle.dump(conditioncode_to_releventcareplanlist, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/careplancode_to_careplandescription.pickle', 'wb') as handle:
    pickle.dump(careplancode_to_description, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/coditioncode_to_listofprocedurescode.pickle', 'wb') as handle:
    pickle.dump(conditioncode_to_releventprocedurelist, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/procedurecode_to_procedurename.pickle', 'wb') as handle:
    pickle.dump(procedurecode_to_description, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/means_coditioncode_observationcode', 'wb') as handle:
    pickle.dump(conditioncode_and_observationcode_to_mean, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/derivation_coditioncode_observationcode.pickle', 'wb') as handle:
    pickle.dump(conditioncode_and_observationcode_to_deviation, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/observationsname_to_code.pickle', 'wb') as handle:
    pickle.dump(observationdescription_to_code, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/procedurecode_to_cost.pickle', 'wb') as handle:
    pickle.dump(procedurecode_to_cost, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/[companycode][conditioncode][procedurecode]_to_ratio.pickle', 'wb') as handle:
    pickle.dump(insurancecompanycode_and_conditioncode_and_procedurecode_to_recoveredratio, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/insucompanyname_to_code.pickle', 'wb') as handle:
    pickle.dump(insurancecompanyname_to_code, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('pickles/insucompanycode_to_name.pickle', 'wb') as handle:
    pickle.dump(insurancecode_to_company, handle, protocol=pickle.HIGHEST_PROTOCOL)